In [1]:
from imp import reload
import convnets; reload(convnets)
from convnets import *

from keras.wrappers.scikit_learn import KerasClassifier
from keras.backend.tensorflow_backend import set_session

from sklearn.metrics import label_ranking_average_precision_score, accuracy_score, f1_score, fbeta_score

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
set_session(tf.Session(config=config))

keras.backend.image_dim_ordering = 'tf'

/home/ubuntu/anaconda3/envs/deepenv/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/deepenv/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# Planet: satellite image competition
In this notebook, transfer learning is used with vgg16. The top layers following the last convolutional layer are retrained. Data Augmentation is applied on train images. The performance metrics is the f2-score.

## Preparing data
- resizing all umages to 224x224 (i.e. VGG16 input shape)
- convert them all to mode RGB
- split the data into train and validation (using stratified k-folds)

### Resizing and reshaping images

In [2]:
path = './train/images/'
listdir = os.listdir(path)

In [3]:
%%time
tmp = dict()
for f in listdir:
    id_ = f.split('.jpg')[0]
    img = imread(path + f)
    img = imresize(img,(224,224,3))
    tmp[id_] = dict(image=img)

CPU times: user 1min 55s, sys: 2.74 s, total: 1min 58s
Wall time: 1min 57s


### Stratified K-folds split

In [4]:
dt = pd.read_csv('train_v2.csv')

In [5]:
labels = dt.tags.apply(lambda l: l.split(' ')).values

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

df = pd.DataFrame(mlb.fit_transform(labels),columns=[mlb.classes_])
df['id_'] = dt.image_name.values ; df['labels'] = labels
df.shape

(40479, 19)

In [7]:
mlb.classes_

array(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming',
       'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation',
       'habitation', 'haze', 'partly_cloudy', 'primary', 'road',
       'selective_logging', 'slash_burn', 'water'], dtype=object)

In [8]:
from sklearn import preprocessing
import random

le = preprocessing.LabelEncoder()

X = df.id_.as_matrix()
y = df[mlb.classes_].as_matrix()
y_encoded = le.fit_transform(df['labels'].apply(lambda i: random.choice(i)))

X.shape, y.shape, y_encoded.shape

((40479,), (40479, 17), (40479,))

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit
skf = StratifiedShuffleSplit(n_splits=1, random_state=42)

In [10]:
for train_index, test_index in skf.split(X=X ,y=y_encoded): 
    
    X_train = np.array([tmp.get(k)['image'] for k in X[train_index]])
    X_test = np.array([tmp.get(k)['image'] for k in X[test_index]])
    
    y_train = y[train_index]
    y_test = y[test_index]

### Data augmentation

In [15]:
gen=image.ImageDataGenerator(zoom_range=0.1,
                             horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.05,
                             height_shift_range=0.05)
shuffle=True ; batch_size=64 ; class_mode='categorical'; target_size=(224,224); epochs = 20

## Training model

In [16]:
model = top_model_vgg_multi(n_classes=17)
model.fit_generator(gen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs,validation_data=(X_test,y_test))

Epoch 1/20
570/569 [==============================] - 644s - loss: 0.1731 - val_loss: 0.1194
Epoch 2/20
570/569 [==============================] - 639s - loss: 0.1352 - val_loss: 0.1133
Epoch 3/20
570/569 [==============================] - 644s - loss: 0.1303 - val_loss: 0.1120
Epoch 4/20
570/569 [==============================] - 637s - loss: 0.1278 - val_loss: 0.1096
Epoch 5/20
570/569 [==============================] - 678s - loss: 0.1266 - val_loss: 0.1083
Epoch 6/20
570/569 [==============================] - 733s - loss: 0.1246 - val_loss: 0.1111
Epoch 7/20
570/569 [==============================] - 737s - loss: 0.1233 - val_loss: 0.1071
Epoch 8/20
570/569 [==============================] - 746s - loss: 0.1226 - val_loss: 0.1069
Epoch 9/20
570/569 [==============================] - 646s - loss: 0.1213 - val_loss: 0.1051
Epoch 10/20
570/569 [==============================] - 729s - loss: 0.1213 - val_loss: 0.1056
Epoch 11/20
570/569 [==============================] - 726s - loss: 0

In [13]:
model = top_model_vgg_multi(n_classes=17)
model.save_weights('vgg_sat.h5')
#model.load_weights('vgg_sat.h5')

### Validation
##### without finetuning the probability threshold (default = 0.5)

In [12]:
tmp = dict()
probas = model.predict(X_test,verbose=1)
classes = np.round(probas).astype('int')

tmp['precision'] = label_ranking_average_precision_score(y_test,probas)
tmp['acc'] = accuracy_score(y_test,classes)
tmp['f1'] = f1_score(y_test, classes,average='samples')
tmp['fbeta'] = fbeta_score(y_test,classes,average='samples',beta=2)

print('**************************************************************************')
print('accuracy:', tmp['acc'], '-- LRAP', tmp['precision'], '-- f1 score:', tmp['f1'], '-- fbeta:',tmp['fbeta'])
print('**************************************************************************')

4048/4048 [==============================] - 76s    
**************************************************************************
accuracy: 0.619812252964 -- LRAP 0.958481061981 -- f1 score: 0.893545399758 -- fbeta: 0.883856410629
**************************************************************************


/home/ubuntu/anaconda3/envs/deepenv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


##### Finetuning the probability threshold

In [30]:
probas = model.predict(X_test,verbose = 1)

4048/4048 [==============================] - 71s    


In [31]:
list_scores = []

for th in [0.15,0.2,0.25,0.35,0.5]:
    tmp = dict()
    classes = probas
    for idx, e in enumerate(classes):
        classes[idx] = [0 if pr < th else 1 for pr in e]
        classes = classes.astype(int)
        
    tmp['precision'] = label_ranking_average_precision_score(y_test,probas)
    tmp['acc'] = accuracy_score(y_test,classes)
    tmp['f1'] = f1_score(y_test, classes,average='samples')
    tmp['fbeta'] = fbeta_score(y_test,classes,average='samples',beta=2)

    print('**************************************************************************')
    print('accuracy:', tmp['acc'], '-- LRAP', tmp['precision'], '-- f1 score:', tmp['f1'], '-- fbeta:',tmp['fbeta'])
    print('**************************************************************************')
    list_scores.append(tmp)

**************************************************************************
accuracy: 0.467144268775 -- LRAP 0.958465254314 -- f1 score: 0.859230044494 -- fbeta: 0.909311643599
**************************************************************************
**************************************************************************
accuracy: 0.511363636364 -- LRAP 0.958465254314 -- f1 score: 0.87690237957 -- fbeta: 0.912163996
**************************************************************************
**************************************************************************
accuracy: 0.545701581028 -- LRAP 0.958465254314 -- f1 score: 0.88707295943 -- fbeta: 0.911067484733
**************************************************************************
**************************************************************************
accuracy: 0.604249011858 -- LRAP 0.958465254314 -- f1 score: 0.89802270221 -- fbeta: 0.905581688851
**************************************************************************
**

/home/ubuntu/anaconda3/envs/deepenv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


## Predicting
### test set 1

In [32]:
dtest = pd.read_csv('submit.csv')
test = dtest.image_name[:40669].values
test_add = dtest.image_name[40669:].values

In [33]:
%%time
tmp = dict()
path = './test/images/'
for f in test:
    img = imread(path + f + '.jpg',mode='RGB')
    img = imresize(img,(224,224,3))
    tmp[f] = dict(image=img)

CPU times: user 1min 57s, sys: 184 ms, total: 1min 57s
Wall time: 1min 56s


In [34]:
X_sub = np.array([tmp.get(k)['image'] for k in test])

In [35]:
probas = model.predict(X_sub,verbose=1)
preds = probas
th=0.2
for idx, e in enumerate(preds):
    preds[idx] = [0 if pr < th else 1 for pr in e]
    preds = preds.astype(int)
preds = [list(i) for i in mlb.inverse_transform(preds)]
preds = [' '.join(map(str, i)) for i in preds]

40669/40669 [==============================] - 716s   


In [36]:
len(preds)

40669

### test set 2

In [38]:
%%time
tmp_add = dict()
path = './test_add/images_add/'
for f in test_add:
    img = imread(path + f + '.jpg',mode='RGB')
    img = imresize(img,(224,224,3))
    tmp_add[f] = dict(image=img)

CPU times: user 59.1 s, sys: 1.33 s, total: 1min
Wall time: 1min


In [39]:
X_sub_add = np.array([tmp_add.get(k)['image'] for k in test_add])

In [ ]:
probas_add = model.predict(X_sub_add,verbose=1)
preds_add = probas_add
th=0.2
for idx, e in enumerate(preds_add):
    preds_add[idx] = [0 if pr < th else 1 for pr in e]
    preds_add = preds_add.astype(int)
preds_add = [list(i) for i in mlb.inverse_transform(preds_add)]
preds_add = [' '.join(map(str, i)) for i in preds_add]

 4960/20522 [======>.......................] - ETA: 274s

In [43]:
len(preds_add)

20522

### merging predictions

In [45]:
merged_preds = preds + preds_add
merged_probas = [i for i in probas] + [j for j in probas_add]
to_submit = pd.DataFrame({'image_name': dtest.image_name.values,
                          'tags':merged_preds})
np.save('vgg_probas_v2',merged_probas)

In [46]:
to_submit.to_csv('submission_v2.csv',index=False)